In [ ]:
import csv
from sentence_transformers import SentenceTransformer
import numpy
from cleantext import clean
import sqlite3
import os
import csv
import json




/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:

def getvideos(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT videoid, payload FROM yt_videos WHERE status > 0 AND lang = 'en'")
    videos = []
    for videoid, payload in cursor.fetchall():
        data = json.loads(payload)
        title = data.get('title')
        description = data.get('description')
        if title is not None:
            videos.append((videoid, title, description))
    cursor.close()
    return videos

In [ ]:
def dbconnection():
    # Connect to a local SQLite database file
    conn = sqlite3.connect('fitlabeler.db')
    return conn

In [ ]:
data = getvideos(dbconnection())
f = open("../data/data.csv", "w")
wr = csv.writer(f, delimiter=",")
wr.writerows(data)

In [5]:
df = pd.read_csv('/kaggle/input/yt-fitness-v1/data.csv',header=None)

In [7]:
df['docs'] = df[1]+df[2]

In [8]:
def clean_doc(x):
    clean_doc=clean(x,
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=True,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=False,                 # remove punctuations
    no_emoji=True,
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en"                       # set to 'de' for German special handling
         )
    clean_doc = clean_doc.replace('"','').replace('null','').replace('#','')
    return clean_doc

In [9]:
df['clean_docs'] = df['docs'].apply(clean_doc)

In [10]:
df['length'] = df['clean_docs'].apply(lambda x: len(x.split()))

In [11]:
df.rename(columns={0:"videoid"},inplace=True)

In [12]:
df.drop(columns={1,2},inplace=True)

In [13]:
df.drop(columns={'docs'},inplace=True)

In [14]:
df = df[df['length'] > 2]

In [ ]:
df.to_csv('../data/Data_out.csv')

In [16]:

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(df['clean_docs'].to_list(), show_progress_bar=False)



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
numpy.savez_compressed('../data/embeddings.csv',embeddings)